# `/test` LangGraph Notebook (Prompt-Synced)

This notebook defines the LangGraph loop like `autonomous_4_agents_langgraph.ipynb`, but **uses the exact same agent-node implementations and prompts as `/test`** by importing them from `app.agent`.

This guarantees prompt parity with the running app.


## 0. Optional Install

In [ ]:
# !pip install -q langgraph langchain-core langchain-groq arxiv chromadb sentence-transformers python-dotenv pypdf2


## 1. Imports

In [ ]:
import json
from datetime import datetime
from pprint import pprint

from langgraph.graph import StateGraph, END

from app.agent import (
    AgentState,
    create_new_session,
    planner_agent,
    researcher_agent,
    evaluator_agent,
    expansion_agent,
    RESEARCHER_PROMPT,
    EVALUATOR_PROMPT,
    save_results,
    generate_report,
)


## 2. Prompt Parity Check
The prompts below are imported directly from `app.agent`.


In [ ]:
print("Researcher prompt (first 400 chars):")
print(RESEARCHER_PROMPT[:400])
print("
Evaluator prompt:")
print(EVALUATOR_PROMPT)


## 3. Build Full Graph (in notebook)
Graph structure is the same loop used in backend mission execution.


In [ ]:
AGENT_REGISTRY = {
    "Planner": planner_agent,
    "Researcher": researcher_agent,
    "Evaluator": evaluator_agent,
    "Expansion": expansion_agent,
}


def agent_executor(state: AgentState) -> AgentState:
    current = state.get("current_agent", "Planner")
    fn = AGENT_REGISTRY.get(current)
    if fn is None:
        return {
            **state,
            "next_agent": "END",
            "current_agent": "END",
            "trace": state.get("trace", []) + [f"{current} → UNKNOWN → END"],
        }

    update = fn(state)
    next_agent = update.get("next_agent", "END")
    trace = state.get("trace", []) + [f"{current} → {next_agent}"]
    return {
        **state,
        **update,
        "current_agent": next_agent,
        "trace": trace,
    }


def route(state: AgentState) -> str:
    return "END" if state.get("next_agent", "END") == "END" else "LOOP"


def build_graph():
    g = StateGraph(AgentState)
    g.add_node("agent_executor", agent_executor)
    g.set_entry_point("agent_executor")
    g.add_conditional_edges(
        "agent_executor",
        route,
        {
            "LOOP": "agent_executor",
            "END": END,
        },
    )
    return g.compile()

GRAPH = build_graph()
print("Graph compiled")


## 4. Prepare Mission State (same shape as `/test` backend)


In [ ]:
topic = "AI for climate-resilient agriculture"
search_mode = "online"  # "online" or "workspace"
max_iterations = 3

session_id = create_new_session()

events = []
tokens = []

def on_event(payload: dict):
    events.append(payload)
    step = payload.get("step", "")
    status = payload.get("status", "")
    summary = payload.get("summary", "")
    if status in {"running", "complete", "error"}:
        print(f"[{step}] {status}: {summary}")

def on_token(step: str, agent: str, token: str):
    tokens.append({"step": step, "agent": agent, "token": token})

state = {
    "session_id": session_id,
    "topic": topic,
    "goal": topic,
    "search_mode": search_mode,
    "papers": [],
    "gaps": [],
    "weak_gaps": [],
    "ideas": [],
    "novelty_scores": [],
    "evidence_assessment": {},
    "insufficient_evidence_message": "",
    "insufficient_data": False,
    "insufficiency_reason": "",
    "current_agent": "Planner",
    "next_agent": "Planner",
    "iterations": 0,
    "max_iterations": max_iterations,
    "future_ideas": [],
    "trace": [],
    "_emit": on_event,
    "_token": on_token,
}


## 5. Run Graph

In [ ]:
result = GRAPH.invoke(state)
result.pop("_emit", None)
result.pop("_token", None)

print("
Mission complete")
print("Trace:")
for t in result.get("trace", []):
    print(" -", t)


In [ ]:
print("
Summary:")
print("papers:", len(result.get("papers", [])))
print("strong gaps:", len(result.get("gaps", [])))
print("weak gaps:", len(result.get("weak_gaps", [])))
print("ideas:", len(result.get("ideas", [])))
print("insufficient:", result.get("insufficient_data", False))
print("
Evidence assessment:")
print(json.dumps(result.get("evidence_assessment", {}), indent=2))

if result.get("ideas"):
    print("
First idea:")
    pprint(result["ideas"][0])


## 6. Save Session Output + Generate Report
Uses the same save/report helpers as backend.


In [ ]:
save_update, save_event = save_results(result)
result.update(save_update)
print(save_event.get("summary", "Saved"))
print("Output file:", result.get("output_file"))


In [ ]:
report_md = generate_report(result)
print(report_md[:3000])


## 7. Inspect Stream Events/Tokens

In [ ]:
print("Event count:", len(events))
print("Token chunks:", len(tokens))
print("
Last 5 events:")
for ev in events[-5:]:
    print(json.dumps(ev, ensure_ascii=False)[:400])
